In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with AI21 Labs Models
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/ai21labs_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fgenerative_ai%2Fai21labs_intro.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">                                                                             
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/ai21labs_intro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/ai21labs_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  
</table>

## Overview

### AI21 Labs on Vertex AI

AI21 Labs models on Vertex AI offer fully managed and serverless models as managed APIs. To use an AI21 on Vertex AI, send a request directly to the Vertex AI API endpoint.

You can stream your AI21 models responses to reduce the end-user latency perception. A streamed response uses server-sent events (SSE) to incrementally stream the response.

### Available AI21 Labs models

#### Jamba 1.5 Mini
AI21's small, powerful instruction-tuned foundation model with 256K context window that's optimized for long-form input, speed and cost efficiency.

#### Jamba 1.5 Large
AI21's most powerful instruction-tuned foundation model with 256K context window that's optimized for long-form input, superior accuracy, and speed.

## Objective

This notebook demonstrates how to use the **Vertex AI API** to access the AI21 Jamba 1.5 Mini and Jamba 1.5 Large models on Vertex AI.

For more information, see the [Use AI21 Labs](https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/ai21) documentation.


## Vertex AI API

## Get Started


### Install required packages


In [ ]:
! pip3 install -U -q httpx

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

#### Select one of AI21 Labs models

In [ ]:
MODEL = "jamba-1.5-mini"  # @param ["jamba-1.5-mini", "jamba-1.5-large" ]

if MODEL == "jamba-1.5-mini":
    available_regions = ["us-central1", "europe-west4"]
    available_versions = ["latest", "001"]

elif MODEL == "jamba-1.5-large":
    available_regions = ["us-central1", "europe-west4"]
    available_versions = ["latest", "001"]

#### Select a location and a version from the dropdown

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown_loc = widgets.Dropdown(
    options=available_regions,
    description="Select a location:",
    font_weight="bold",
    style={"description_width": "initial"},
)

dropdown_ver = widgets.Dropdown(
    options=available_versions,
    description="Select the model version (optional):",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_loc_eventhandler(change):
    global LOCATION
    if change["type"] == "change" and change["name"] == "value":
        LOCATION = change.new
        print("Selected:", change.new)


def dropdown_ver_eventhandler(change):
    global MODEL_VERSION
    if change["type"] == "change" and change["name"] == "value":
        MODEL_VERSION = change.new
        print("Selected:", change.new)


LOCATION = dropdown_loc.value
dropdown_loc.observe(dropdown_loc_eventhandler, names="value")
display(dropdown_loc)

MODEL_VERSION = dropdown_ver.value
dropdown_ver.observe(dropdown_ver_eventhandler, names="value")
display(dropdown_ver)

#### Set Google Cloud project and model information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"
SELECTED_MODEL_VERSION = "" if MODEL_VERSION == "latest" else f"@{MODEL_VERSION}"

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")

### Text generation

#### Unary call

Sends a POST request to the specified API endpoint to get a response from the model for a joke using the provided payload.

In [ ]:
import json

PAYLOAD = {
    "model": MODEL,
    "messages": [{"role": "user", "content": "Tell me a joke about whales"}],
    "max_tokens": 100
}

request = json.dumps(PAYLOAD)

!curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/ai21/models/{MODEL}{SELECTED_MODEL_VERSION}:rawPredict \
  -d '{request}'

#### Streaming call

Sends a POST request to the specified API endpoint to stream a response from the model for a sports T-Shirt product title using provided payload.

In [ ]:
import json

PAYLOAD = {
    "model": MODEL,
    "messages": [{"role": "user", "content": "Write a product title for a sports T-Shirt to be published for online retail. Include these keywords: activewear, gym, dryfit."}],
    "max_tokens": 100
}

request = json.dumps(PAYLOAD)
!curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/ai21/models/{MODEL}{SELECTED_MODEL_VERSION}:streamRawPredict \
  -d '{request}'